In [1]:
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
path_to_db = '/datasets/telecomm_csi.db'
engine = create_engine(f'sqlite:///{path_to_db}', echo = False)

In [3]:
query = """
SELECT * FROM location;
"""

In [4]:
df = pd.read_sql(query, engine)
df.head(3)

,location_id,city,country
0,1,Архангельск,Россия
1,2,Астрахань,Россия
2,3,Балашиха,Россия


In [5]:
query = """
SELECT * FROM user;
"""

In [6]:
df_user = pd.read_sql(query, engine)
df_user.head(3)

,user_id,lt_day,age,gender_segment,os_name,cpe_type_name,location_id,age_gr_id,tr_gr_id,lt_gr_id,nps_score
0,A001A2,2320,45.0,1.0,ANDROID,SMARTPHONE,55,5,5,8,10
1,A001WF,2344,53.0,0.0,ANDROID,SMARTPHONE,21,5,5,8,10
2,A003Q7,467,57.0,0.0,ANDROID,SMARTPHONE,28,6,9,6,10


In [7]:
#df.to_csv('telecomm_csi_tableau.csv', index=False)

In [8]:
query ="""
SELECT u.user_id,
    u.lt_day, CASE
    WHEN lt_day < 365 THEN "new"
    ELSE "old"
    END AS is_new,
    u.gender_segment, CASE
    WHEN gender_segment = 1 THEN'female'
    ELSE 'male' 
    END AS gender_segment,
    u.age,
    u.os_name,
    u.cpe_type_name,
    l.country,
    l.city,
    as2.title as age_segment,
    ts.title as traffic_segment,
    ls.title as lifetime_segment,
    u.nps_score,CASE
    WHEN u.nps_score BETWEEN 9 AND 10 THEN 'promoters'
    WHEN u.nps_score BETWEEN 7 AND 8 THEN 'passives'
    ELSE 'detractors'
    END as nps_group
FROM user u
LEFT JOIN traffic_segment ts 
ON u.tr_gr_id= ts.tr_gr_id
LEFT JOIN location l 
ON u.location_id=l.location_id
LEFT JOIN age_segment as2 
ON u.age_gr_id = as2.age_gr_id
LEFT JOIN lifetime_segment ls 
ON u.lt_gr_id =ls.lt_gr_id      
"""

In [9]:
data = pd.read_sql(query, engine)
data.head(5)

,user_id,lt_day,is_new,gender_segment,gender_segment,age,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,old,1.0,female,45.0,ANDROID,SMARTPHONE,Россия,Уфа,05 45-54,04 1-5,08 36+,10,promoters
1,A001WF,2344,old,0.0,male,53.0,ANDROID,SMARTPHONE,Россия,Киров,05 45-54,04 1-5,08 36+,10,promoters
2,A003Q7,467,old,0.0,male,57.0,ANDROID,SMARTPHONE,Россия,Москва,06 55-64,08 20-25,06 13-24,10,promoters
3,A004TB,4190,old,1.0,female,44.0,IOS,SMARTPHONE,Россия,РостовнаДону,04 35-44,03 0.1-1,08 36+,10,promoters
4,A004XT,1163,old,0.0,male,24.0,ANDROID,SMARTPHONE,Россия,Рязань,02 16-24,05 5-10,08 36+,10,promoters


In [10]:
data.to_csv('telecomm_csi_tableau.csv', index=False)